### This file is the draft of bootstrapping bunching estimation.

We calculate the Variance of the Dd and Frac_nb based on the original boostrap process.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import copy 
import random
from matplotlib import pyplot as plt
from numpy.linalg import inv
from scipy.optimize import minimize
from scipy.stats import norm

%matplotlib inline

path = '/Users/philhuang/Desktop/FDI_emp/workingdata/'
output_folder = '/Users/philhuang/Desktop/FDI_emp/tempdata/bunching_bootstrap/'

In [2]:
# Read in the data.
ASIF_df = pd.read_csv(path+'ASIF_panel_bootstrapBucnhing.csv')

# Look at the data.
ASIF_df

,year,panel_id,share_fdi,bin_id
0,1998,SZ10000019,0.509798,50
1,1999,SZ10000019,0.509798,50
2,2000,SZ10000019,0.509798,50
3,2001,SZ10000019,0.509798,50
4,2002,SZ10000019,0.509798,50
...,...,...,...,...
150761,2005,SZ10890785,0.925039,92
150762,2000,SZ10890801,0.693488,69
150763,2001,SZ10890801,0.699574,69
150764,2002,SZ10890801,0.699574,69


In [3]:
# Number of unique panel_id for the original sample.
panel_id_lst = pd.DataFrame(ASIF_df.reset_index().panel_id.unique())
orgianl_id_num = len(panel_id_lst)
orgianl_id_num

43063

#### Define a function to generate a bootstrap sample from the original sample.

In [4]:
def bootstrap_ASIF(seed_num):
    
    # Bootstrap panel_id with replacement, and get a list of panel_id of the new sample.
    bootstrap_id = panel_id_lst.sample(n=orgianl_id_num, random_state=seed_num, replace=True)
    bootstrap_id.rename(columns={0:'panel_id'}, inplace=True)

    # Merge the new list of panel_id with the original sample to form a new bootstrapped sample.
    rst = bootstrap_id.merge(ASIF_df, on=['panel_id'], how='left')
    
    return rst

#### Define a function to get the distribution of FDI share by bin, and generate training and testing data.

In [5]:
def get_test_train_boots(target_df, random_state):
    #### Calculate the vars we need for the whole sample.
    # Calculate the number of firms in each bin.
    overall_bin_count = np.array(target_df.groupby('bin_id')['panel_id'].count())
    overall_bin_count

    # Calculate the total of all firms.
    total_firms = np.sum(overall_bin_count)

    # Calculate the prob of firms falling into each bin.
    overall_bin_prob = overall_bin_count/total_firms

    #### Generate test and training data. 
    # Generate random numbers between [0,1]
    np.random.seed(random_state)
    target_df['random_num'] = np.random.uniform(0,1, total_firms)
    # Generate ID of the train group based on the random_num. Note that we split the data as follows:
    # 20% testing and 80% training.
    target_df['test_group_id'] = 1
    target_df.loc[target_df['random_num']>=0.2, 'test_group_id'] = 2
    target_df.loc[target_df['random_num']>=0.4, 'test_group_id'] = 3
    target_df.loc[target_df['random_num']>=0.6, 'test_group_id'] = 4
    target_df.loc[target_df['random_num']>=0.8, 'test_group_id'] = 5

    # Store the results.
    train_data = []
    test_data  = []

    for i in range(1,6):
        # Take out each test group.
        temp_df     = target_df[target_df['test_group_id']==i]
        # Calculate the prob of firms falling into each bin in the test group.
        temp_prob_bin = np.array(temp_df.groupby('bin_id')['panel_id'].count()/len(temp_df))
        # Store the result to the list.
        test_data.append(temp_prob_bin)

        # Take out each training group.
        temp_df     = target_df[target_df['test_group_id']!=i]
        # Calculate the prob of firms falling into each bin in the test group.
        temp_prob_bin = np.array(temp_df.groupby('bin_id')['panel_id'].count()/len(temp_df))
        # Store the result to the list.
        train_data.append(temp_prob_bin)

    # Return things in a DataFrame
    df_start                  = pd.DataFrame(np.array(list(range(0,100)))/100, columns=['bin'])
    df_start['count']         = overall_bin_prob
    df_start['count_train_1'] = train_data[0]
    df_start['count_train_2'] = train_data[1]
    df_start['count_train_3'] = train_data[2]
    df_start['count_train_4'] = train_data[3]
    df_start['count_train_5'] = train_data[4]
    df_start['count_test_1']  = train_data[0]
    df_start['count_test_2']  = train_data[1]
    df_start['count_test_3']  = train_data[2]
    df_start['count_test_4']  = train_data[3]
    df_start['count_test_5']  = train_data[4]

    return df_start

#### Now estimate bunching using the dataset we have at hand. 

In [6]:
# Define the grp2idx function that we will use later in the find_Dm function
# This is equivalent to grp2idx function in Matlab
# This convert [0,0,1,1,1,0,0] to [1,1,2,3,4,1,1]
def grp2idx(lst):
    rst = []
    category = 1
    for i in range(len(lst)):
        if lst[i] == 0:
            rst.append(1)
        else:
            category += 1
            rst.append(category)
    return np.array(rst)

In [7]:
# This function constructs w_matrix, which is the polynomials and fixed effect;
# df_dx, which is the first order derivative w.r.t x of w_matrix
# db, which is just a vector of 0.

def construct_matrix(minD, p, Dp, Dm, X, bins):
    # This part adds round number fixed effects to the regression
    round_num_lst = np.array(list(range(25, 100, 5)))/100
    len_round_num = 3                                            # We only need three round-num dummy here. Including 0.5 fixed effect.
    dummy_5=[]
    dummy_10=[]
    dummy_50=[]
    for item in all_bins[all_bins>minD]:
        if item in round_num_lst and int(item*100)%10 ==5:
            dummy_5.append(1)
        else:
            dummy_5.append(0)
    for item in all_bins[all_bins>minD]:
        if item in round_num_lst and int(item*100)%10 ==0:
            dummy_10.append(1)
        else:
            dummy_10.append(0)
    for item in all_bins[all_bins>minD]:
        if item == 0.5:
            dummy_50.append(1)
        else:
            dummy_50.append(0)
            
    round_num_FE = pd.DataFrame({'dummy_5':dummy_5, 'dummy_10':dummy_10, 'dummy_50':dummy_50})
    
    # This part adds bunching fixed effect to the regression (the excluded region). 
    group = bins[bins>minD] * ((bins[bins>minD] > Dm) & (bins[bins>minD] < Dp))
    group = grp2idx(group)
    group_df = pd.DataFrame(data = group, columns=['group'])
    FE = pd.get_dummies(group_df.group)
    w = pd.concat([X, round_num_FE, FE.iloc[:,1:]], axis=1)      # Here we get p-polynomials of the edges and fixed effect
    w.columns=list(range(len(w.columns)))                        # between Dm and Dp
    w_matrix = np.array(w)
    
    # This part calculates the derivative matrix.
    const = ((bins[bins>minD] > Dm) & (bins[bins>minD] < notch)).astype(int)
    dx=[0*const, const]
    for j in range(2,p+1):
        dx.append(np.array(j*const*bins[bins>minD]**(j-1)))      # gradient of the polynomials
    df_dx = pd.DataFrame(data=np.array(dx).T)
    df_dx = pd.concat([df_dx, 0*round_num_FE, 0*FE.iloc[:,1:]], axis=1)
    df_dx.columns=list(range(len(df_dx.columns)))
    df_dx = np.array(df_dx)
    
    db = np.zeros(len(all_bins[all_bins>minD]))                                 
    
    return w_matrix, df_dx, db, len_round_num


In [8]:
# This function solves for the optimal b under the constraint

def get_beta(w_matrix, C_minD_end, df_dx):
        b0 = np.linalg.lstsq(w_matrix, C_minD_end, rcond=None)[0]    # b0 is the unconstrained beta. Now the have 76 equations
                                                                     # but 28 parameters, we have to use ols to estimate b
        def objective(b):                                            # Calculate objective f
            resid = C_minD_end-w_matrix.dot(b)
            f = (resid.T).dot(resid)
            return f

        def constraint1(b):                                          # note in Python scipy.minimize, the constraint should be f(x)>=0,
            g = -df_dx.dot(b)                                        # so we set g to be the opposite of the F.O.C
            return g

        cons1 = {'type':'ineq', 'fun': constraint1}
        #sol = minimize(objective, b0, method='SLSQP', constraints=cons1)
        sol = minimize(objective, b0, method='SLSQP')

        b = sol.x
        return b 
    

In [9]:
# Define a function to find the lower bound of bunching.
def find_Dm(minD, p, Dp, X, C, bins, notch, nReps):
    results = []
    C_minD_end = C[bins>minD]
    d = bins[1] - bins[0]
    lst_Dm = bins[(bins>(minD*1.0+4*d)) & (bins < (notch*1.0-8*d))] # all possible Dm

    for i in lst_Dm:
        Dm = i
        ## Here the w matrix: 25 with FE, but not round number effect 
        w_matrix, df_dx, db, len_round_num = construct_matrix(minD, p, Dp, Dm, X, bins)
        b = get_beta(w_matrix, C_minD_end, df_dx)

        ## Here the w_matrix: 25 with FE, but not round number effect 
        criterion_Dm_Dp= ((bins>Dm) & (bins<Dp))[-len(w_matrix):]     
        results.append([Dm, 
                        np.sum(w_matrix[criterion_Dm_Dp][:,:p+len_round_num+1].dot(b[:p+len_round_num+1])         ## What we get here is negative FE: polynomial + Rnum effect - the observed data
                               - w_matrix[criterion_Dm_Dp].dot(b)),                                               ## This part has FE, which is a perfect fit of the observed data 
                        np.sum(w_matrix[criterion_Dm_Dp][:,:p+len_round_num+1].dot(b[:p+len_round_num+1])),       ## counterfactual C 
                        np.sum(w_matrix[criterion_Dm_Dp][:, p+len_round_num+1:].dot(b[p+len_round_num+1:]))])     ## FE(excluded region)
    ## Results
    results_df = pd.DataFrame(np.array(results))
    # There might be multiple candidates, we choose the smallest one. The Dm satisfy: the effct of excluded
    # region FE is the smallest.
    Dm   = results_df[abs(results_df[1]) == abs(results_df[1]).min()][0].min() 
    diff = results_df[results_df[0]==Dm][1].min()                               ## negative FE
    DD   = results_df[results_df[0]==Dm][2].min()

    ## With the optimal Dm, we now get main estimation. Note that here we only use the w matrix for estimation 
    w_matrix, df_dx, db, len_round_num = construct_matrix(minD, p, Dp, Dm, X, bins)
    b = get_beta(w_matrix, C_minD_end, df_dx)
    resid = w_matrix.dot(b) - C_minD_end

    ## Bootstrap b and DD
    id_matrix = np.ceil(np.random.rand(len(resid), nReps)*len(resid))     # get random numbers as the bstrap id. There can be repeated numbers in 
                                                                          # one column,which means we draw the residual with replacement
    diff_b =[]
    for i in range(nReps):
        tempC = C_minD_end + resid[(id_matrix[:,i]-1).astype(int)]          
        btrap_b = get_beta(w_matrix, tempC, df_dx)

        # Note now we have a new Dm (the optimal Dm), we need to specify a new criterion_Dm_Dp
        # Now we use the w matrix for prediction
        criterion_Dm_Dp= ((bins>Dm) & (bins<Dp))[-len(w_matrix):]
        diff_b.append(np.sum(w_matrix[criterion_Dm_Dp][:, :p+len_round_num+1].dot(btrap_b[:p+len_round_num+1]) 
                             - w_matrix[criterion_Dm_Dp][:, p+len_round_num+1:].dot(btrap_b[p+len_round_num+1:])))

    pval = 2*(1- norm.cdf(abs(diff/np.std(diff_b))))
    
    return Dm, pval, DD


In [10]:
# Define a function to calculate SSE, which helps to find the best estimator which minimizes SSE.
def SSE_K(minD, p, Dp, C, df_C, bins, notch, nReps):
    # Generate polynomials
    x = bins[bins>minD]
    xs = []
    for i in range(p+1):
        xs.append(x**i)
    xs=np.array(xs).T
    df_x = pd.DataFrame(data=xs)

    # Find Dm 
    Dm,pval,DD = find_Dm(minD, p, Dp, df_x, C, bins, notch, nReps) 
    w_matrix, df_dx, db, len_round_num= construct_matrix(minD, p, Dp, Dm, df_x, bins)
    
    # Compute SSE for K-Fold CV
    SSE = 0 
    for k in range(1,6):
        C_train = np.array(df_C.iloc[:,k+1])[bins>minD]
        C_test  = np.array(df_C.iloc[:,k+6])[bins>minD]
        b_Kfold = get_beta(w_matrix, C_train, df_dx)
        SSE +=  sum((w_matrix.dot(b_Kfold)- C_test)**2) 
    SSE = 1000*SSE/len(bins[bins>minD])
    return Dm, SSE, pval, DD

In [11]:
# Define a function to calculate p,Dp, minD, Dm, SSE, pval, DD
def get_pDpminDDm(work_df, notch, nReps):

    # Fetch the information of bin-cut threshold and the number of firms in each bin.
    #all_bins = np.around(np.array(work_df['bin']), 3)
    #C_all = np.array(work_df['count'])

    # Generate Combinations of (p,Dp,minD) 
    param = []
    lst_Dp = all_bins[(all_bins > notch*1.05) & (all_bins <=0.295)]
    #lst_Dp = all_bins[(all_bins > notch*1.15) & (all_bins <0.04)]
    lst_minD = all_bins[(all_bins > all_bins[0]) & (all_bins <=0.02)]
    #lst_minD = all_bins[(all_bins > all_bins[0]) & (all_bins <=0.003)]
    lst_p = [3,4]

    for p in lst_p:
        for Dp in lst_Dp:
            for minD in lst_minD:
                param.append([p,Dp,minD])

    # Store things in a dataFrame.
    df_param = pd.DataFrame(data =param, columns=['p', 'Dp', 'minD'])

    # Create a list to store all the SSE of different combinations of p, Dp and minD.
    SSE_out = []
    for p in lst_p:
        for Dp in lst_Dp:
            for minD in lst_minD:
                Dm, SSE, pval, DD = SSE_K(minD, p, Dp, C_all, work_df, all_bins, notch, nReps)
                SSE_out.append([p, Dp, minD, Dm, SSE, pval, DD])

    # Convert it into a dataFrame.
    SSE_out_df = pd.DataFrame(np.array(SSE_out), columns=['p', 'Dp', 'minD', 'Dm', 'SSE', 'pval', 'DD'])

    ## Choose (p,Dp,minD) that minimizes SSE.
    sub_SSE_out  = SSE_out_df[SSE_out_df['pval']>0.1].copy()              # pval should be >0.1
    model_choice = sub_SSE_out.iloc[np.argmin(sub_SSE_out['SSE']),:]      # choose the params with the smallest SSE
    p            = int(model_choice[0])
    Dp           = float(model_choice[1])
    minD         = float(model_choice[2])
    Dm           = float(model_choice[3])
    SSE          = float(model_choice[4])
    pval         = float(model_choice[5])
    DD           = float(model_choice[6])
    
    return p,Dp, minD, Dm, SSE, pval, DD

In [12]:
def Model_Est(minD, p, Dp, Dm, C, bins, notch, nReps):
    
    # Drop all bins that are to the left of the minD bin.
    C_minD_end = C[bins>minD]
    
    # Generate polynomials
    x = bins[bins>minD]
    xs = []
    for i in range(p+1):
        xs.append(x**i)
    xs=np.array(xs).T
    df_x = pd.DataFrame(data=xs)
    
    w_matrix, df_dx, db, len_round_num = construct_matrix(minD, p, Dp, Dm, df_x, bins)  # w matrix for estimate 
    b = get_beta(w_matrix, C_minD_end, df_dx)

    resid = w_matrix.dot(b) - C_minD_end
    
    # Calculate the counterfactuual distribution of firms.
    h = w_matrix[:,:p+len_round_num+1].dot(b[:p+len_round_num+1])                       # Polynomial + Rnum effect 
                                                                                        # w matrix for prediction
    # Bootstrap b and DD 
    # We draw the residuals.
    id_matrix = np.ceil(np.random.rand(len(resid), nReps)*len(resid))             
    b_B = [] 
    h_B = [] 

    for i in range(nReps):
        # Add the residual to the actual C (prob of firms falling into each bin in the data we observed.)
        tempC = C_minD_end + resid[(id_matrix[:,i]-1).astype(int)] 
        
        # Estimate Beta.
        b_temp = get_beta(w_matrix, tempC, df_dx)                                   # w matrix for estimation
        
        # Calculate the predicted prob of firms falling into each bin.
        h_temp = w_matrix[:,:p+len_round_num+1].dot(b_temp[:p+len_round_num+1])     # w matrix for prediction   
        b_B.append(b_temp)
        h_B.append(h_temp)

    b_B_df = pd.DataFrame(np.array(b_B))
    h_B_df = pd.DataFrame(np.array(h_B))

    b_SE = np.array(b_B_df.std(axis=0))
    V = np.array(b_B_df.cov())
    
    return b, b_SE, V, h, h_B_df

In [13]:
# 6- Compute Stats
def BStat(Dm, notch, Dp, C, h, h_B, bins, d):
    
    # Note that here the dimension does not match 
    # criterion_Dm_notch = ((bins>=Dm) & (bins<notch))[:len(h)]
    # criterion_Dm_Dp    = ((bins>=Dm) & (bins<=Dp))[:len(h)]  
    criterion_Dm_notch0 = ((bins>minD) & (bins<notch))[-len(h):]
    criterion_Dm_notch1 = ((bins>minD) & (bins<=notch))[-len(h):]
    #criterion_Dm_Dp    = ((bins>minD) & (bins<=Dp))[-len(h):] 
    
    # Theoretically, all firms should have bunched to the right. However due to firction cost, some firms still stay to the left 
    # Here we calculate the ratio of real stayers over conterfactual stayers. The higher the ratio, the higher the frition cost 
    frac_nb    = np.sum(C[(bins>minD)&(bins<notch)]) / np.sum(h[criterion_Dm_notch0])       
    frac_nb_B  = np.sum(C[(bins>minD) & (bins<notch)])/np.sum(h_B.iloc[:,criterion_Dm_notch0].copy(), axis=1)
    frac_nb_B  = np.array(frac_nb_B)

    b_Bstat    = bins[(bins>minD) & (bins<=notch)]+d                                     # upper bound of bins
    c_Bstat    = C[(bins>minD) & (bins<=notch)]/ np.sum(C[(bins>minD) & (bins<=notch)])  # real conditional probability
    hh         = h[criterion_Dm_notch1]/np.sum(h[criterion_Dm_notch1])                   # counterfactual conditional probability 

    hh_B = []
    temp1 = h_B.iloc[:,criterion_Dm_notch1].copy()
    temp2 = np.sum(h_B.iloc[:,criterion_Dm_notch1].copy(), axis=1)
    for i in range(len(temp1)):
        hh_B.append((temp1.iloc[i,:]/temp2[i]))
    hh_B_df = pd.DataFrame(np.array(hh_B))
    
    #Dd = b_Bstat.dot(c_Bstat - hh)/(b_Bstat.dot(c_Bstat))                      # distance between y and y_hat 
    Dd = b_Bstat.dot(c_Bstat - hh)/(b_Bstat.dot(hh))                            # distance between y and y_hat 

    
    Dd_B_temp=[]
    for i in range(len(hh_B_df.T.columns)):
        Dd_B_temp.append(c_Bstat-hh_B_df.T.iloc[:,i])
    Dd_B = b_Bstat.dot(np.array(Dd_B_temp).T)/(b_Bstat.dot(c_Bstat))
    
    return Dd, Dd_B, frac_nb, frac_nb_B


In [14]:
# Specify notch point. 
notch          = 0.25 
nReps          = 2000 
d              = 0.01

start_time = datetime.now()

# Then we generate the test and training set.
test_train_df = get_test_train_boots(ASIF_df, 12345)

# Fetch the information of bin-cut threshold and the number of firms in each bin.
all_bins = np.around(np.array(test_train_df['bin']), 3)
C_all    = np.array(test_train_df['count'])

# Estimate some basic bunching parameters.
p,Dp, minD, Dm, SSE, pval, DD = get_pDpminDDm(test_train_df, notch, 50)

# Estimate beta, and also the bootstrapped h.
b,b_SE,V,h,h_B  = Model_Est(minD,p,Dp,Dm,C_all,all_bins,notch, nReps)

# Get bootstrapped Dd and frac_nb.
Dd,Dd_B,frac_nb,frac_nb_B = BStat(Dm, notch, Dp, C_all, h, h_B, all_bins, d)

end_time = datetime.now()
print('Duration: {}'.format(end_time-start_time))

Duration: 0:00:03.110260


In [15]:
bunching_momnet_BSrst = pd.DataFrame(data={'Dd_B':Dd_B, 'frac_nb_B':frac_nb_B})
bunching_momnet_BSrst

,Dd_B,frac_nb_B
0,0.220084,0.404314
1,0.210792,0.363581
2,0.241907,0.416094
3,0.215567,0.381341
4,0.202672,0.488823
...,...,...
1995,0.180549,0.440618
1996,0.234301,0.370485
1997,0.220917,0.517284
1998,0.197192,0.361469


In [16]:
bunching_momnet_BSrst.cov()

,Dd_B,frac_nb_B
Dd_B,0.001202,-0.001867
frac_nb_B,-0.001867,0.007888


In [17]:
bunching_momnet_BSrst.to_csv(output_folder+'bootstrap_bunching12June2023.csv', index=False)

In [18]:
p,Dp, minD, Dm, SSE, pval, DD 

(4,
 0.28,
 0.01,
 0.06,
 0.09182893633940276,
 0.9843804207240385,
 0.2546654364683665)

In [19]:
(Dd_B.mean(), frac_nb_B.mean())

(0.208185626943552, 0.460948376140406)

In [21]:
all_bins

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99])

In [22]:
C_all.shape

(100,)